In [ ]:
!git clone https://github.com/iamjhpark/MIntRec2.0.git
%cd MIntRec2.0

/content
Cloning into 'MIntRec2.0'...
remote: Enumerating objects: 403, done.
remote: Counting objects: 100% (403/403), done.
remote: Compressing objects: 100% (298/298), done.
remote: Total 403 (delta 185), reused 269 (delta 94), pack-reused 0 (from 0)
Receiving objects: 100% (403/403), 2.33 MiB | 7.29 MiB/s, done.
Resolving deltas: 100% (185/185), done.
/content/MIntRec2.0


In [ ]:
# requirements.txt 파일은 개발자의 로컬 경로 포함 및 오래된 버전 명시로 인한 오류 발생
!pip install torch torchvision torchaudio --quiet
!pip install numpy scipy scikit-learn pandas nltk tqdm transformers Pillow addict easydict --quiet

In [4]:
# 최신 PyTorch/Transformers 버전에 맞게 AdamW import 경로 수정
!sed -i "s/from transformers import AdamW, get_linear_schedule_with_warmup/from torch.optim import AdamW\nfrom transformers import get_linear_schedule_with_warmup/" ./methods/single_turn/MAG_BERT/manager.py
!sed -i "s/from transformers import AdamW, get_linear_schedule_with_warmup/from torch.optim import AdamW\nfrom transformers import get_linear_schedule_with_warmup/" ./methods/single_turn/MAG_BERT/manager_ood.py
!sed -i "s/from transformers import AdamW, get_linear_schedule_with_warmup/from torch.optim import AdamW\nfrom transformers import get_linear_schedule_with_warmup/" ./methods/single_turn/TEXT/manager.py
!sed -i "s/from transformers import AdamW, get_linear_schedule_with_warmup/from torch.optim import AdamW\nfrom transformers import get_linear_schedule_with_warmup/" ./methods/single_turn/TEXT/manager_ood.py
!sed -i "s/from transformers import AdamW, get_linear_schedule_with_warmup/from torch.optim import AdamW\nfrom transformers import get_linear_schedule_with_warmup/" ./methods/multi_turn/MULT/manager.py
!sed -i "s/from transformers import AdamW, get_linear_schedule_with_warmup/from torch.optim import AdamW\nfrom transformers import get_linear_schedule_with_warmup/" ./methods/multi_turn/MAG_BERT/manager.py
!sed -i "s/from transformers import AdamW, get_linear_schedule_with_warmup/from torch.optim import AdamW\nfrom transformers import get_linear_schedule_with_warmup/" ./methods/multi_turn/TEXT/manager.py
!find . -type f -name "manager*.py" | xargs sed -i "s/, correct_bias=False//g"

In [5]:
!sed -i 's|/home/sharing/Datasets/MIntRec2.0/in-scope|data/MIntRec2.0/in-scope|g' examples/single_turn/run_mag_bert_baselines.sh
!sed -i 's|/home/sharing/Datasets/MIntRec2.0/out-of-scope|data/MIntRec2.0/out-of-scope|g' examples/single_turn/run_mag_bert_baselines.sh

In [6]:
# 개인 서버 경로를 올바른 상대 경로('data')와 모델 이름('bert-large-uncased')으로 모두 교체
!find ./examples -type f -name "*.sh" | xargs sed -i 's|/home/sharing/Datasets/MIntRec2.0|data|g'
!find ./examples -type f -name "*.sh" | xargs sed -i 's|/home/sharing/disk1/pretrained_embedding/bert/bert-large-uncased|bert-large-uncased|g'
!find . -type f -name "*.py" | xargs sed -i 's|/home/sharing/disk1/pretrained_embedding/bert/bert-large-uncased|bert-large-uncased|g'

In [7]:
!find . -type f -name "*.py" | xargs sed -i 's/^[[:space:]]*self\.ood_detection_func = ood_detection_map\[args\.ood_detection_method\]/#&/'

In [8]:
# 원본 코드는 비디오/오디오 특징을 '폴더'에서 찾으려 하지만, 실제 데이터는 단일 '파일(.pkl)'이므로 파일 경로 생성 로직을 실제 파일 이름('swin_roi.pkl' 등)을 가리키도록 수정
# 또한, OOD 데이터 경로가 잘못 생성되는 문제도 함께 수정
target_file_base = 'data/single_turn/base.py'
with open(target_file_base, 'r', encoding='utf-8') as f:
    original_code = f.read()

modified_code = original_code.replace("video_feats_path = os.path.join(data_args['data_path'], args.video_data_path, args.video_feats_path)", "video_feats_path = os.path.join(data_args['data_path'], 'swin_roi.pkl')")
modified_code = modified_code.replace("audio_feats_path = os.path.join(data_args['data_path'], args.audio_data_path, args.audio_feats_path)", "audio_feats_path = os.path.join(data_args['data_path'], 'wavlm_feats.pkl')")
modified_code = modified_code.replace("ood_data_path = os.path.join(args.data_path, 'out-of-scope', args.ood_dataset)", "ood_data_path = os.path.join(args.data_path, 'out-of-scope')")

with open(target_file_base, 'w', encoding='utf-8') as f:
    f.write(modified_code)

In [9]:
!grep "swin_roi.pkl" data/single_turn/base.py
!grep "wavlm_feats.pkl" data/single_turn/base.py

        video_feats_path = os.path.join(data_args['data_path'], 'swin_roi.pkl')
        audio_feats_path = os.path.join(data_args['data_path'], 'wavlm_feats.pkl')


In [10]:
RUN_PY_PATCH = "import sys\\nimport os\\nsys.path.append(os.getcwd())\\n\\n"
!sed -i "1s;^;{RUN_PY_PATCH};" run_single_turn.py

In [11]:
# ===============================================================
# 문제 해결: Text-only 모델 실행 시 발생하는 KeyError 수정
# ===============================================================
print("KeyError 해결을 위해 'configs/single_turn/base.py' 파일을 패치합니다...")

# 수정할 파일 경로
target_file_config_base = 'configs/single_turn/base.py'

# 파일의 전체 내용을 읽어옵니다.
with open(target_file_config_base, 'r', encoding='utf-8') as f:
    original_code = f.read()

# 문제가 되는 기존 ParamManager 클래스 정의를 찾습니다.
# (코드의 들여쓰기와 공백을 정확하게 일치시키는 것이 중요합니다)
original_class_def = """class ParamManager:

    def __init__(self, args):

        args.text_pretrained_model = pretrained_models_path[args.text_backbone]
        args.video_feats_path = video_feats_path[args.video_feats]
        args.audio_feats_path = audio_feats_path[args.audio_feats]

        self.args = EasyDict(dict(vars(args)))"""

# if문을 추가하여 텍스트 전용 모델일 경우 비디오/오디오 설정을 건너뛰도록
# 수정된 ParamManager 클래스 정의를 만듭니다.
replacement_class_def = """class ParamManager:

    def __init__(self, args):

        args.text_pretrained_model = pretrained_models_path[args.text_backbone]

        # 만약 실행하는 모델이 텍스트 전용 모델이 아니라면, 비디오/오디오 경로를 설정합니다.
        if args.method not in ['text', 'text_ood']:
            args.video_feats_path = video_feats_path[args.video_feats]
            args.audio_feats_path = audio_feats_path[args.audio_feats]

        self.args = EasyDict(dict(vars(args)))"""

# 파일 내용에서 원본 코드를 수정된 코드로 교체합니다.
modified_code = original_code.replace(original_class_def, replacement_class_def)

# 수정된 내용으로 파일을 다시 씁니다.
with open(target_file_config_base, 'w', encoding='utf-8') as f:
    f.write(modified_code)

print("\n패치가 완료되었습니다!")
print("이제 'run_text_ood_baselines.sh' 또는 관련 python 명령어를 다시 실행하시면 됩니다.")

# (선택적 검증) 수정이 잘 되었는지 확인
print("\n--- 수정된 ParamManager 클래스 ---")
!grep -A 8 "class ParamManager:" configs/single_turn/base.py
print("-----------------------------------")

KeyError 해결을 위해 'configs/single_turn/base.py' 파일을 패치합니다...

패치가 완료되었습니다!
이제 'run_text_ood_baselines.sh' 또는 관련 python 명령어를 다시 실행하시면 됩니다.

--- 수정된 ParamManager 클래스 ---
class ParamManager:
    
    def __init__(self, args):
        
        args.text_pretrained_model = pretrained_models_path[args.text_backbone]
        args.video_feats_path = video_feats_path[args.video_feats]
        args.audio_feats_path = audio_feats_path[args.audio_feats]
        
        self.args = EasyDict(dict(vars(args)))   
-----------------------------------


In [12]:
# ===============================================================
# 문제 해결: Text-only 모델 실행 시 DataManager에서 발생하는 KeyError 수정
# ===============================================================
print("KeyError 재발생 해결을 위해 'data/single_turn/base.py' 파일을 다시 패치합니다...")

# 수정할 파일 경로
target_file_data_base = 'data/single_turn/base.py'

# 파일의 전체 내용을 읽어옵니다.
with open(target_file_data_base, 'r', encoding='utf-8') as f:
    original_code = f.read()

# 문제가 되는 DataManager 클래스의 __init__ 함수 부분을 찾습니다.
original_init_def = """    def __init__(self, args):

        self.logger = logging.getLogger(args.logger_name)

        bm = benchmarks[args.dataset]
        max_seq_lengths = bm['max_seq_lengths']
        feat_dims = bm['feat_dims']

        args.text_seq_len = max_seq_lengths['text']
        args.video_seq_len = max_seq_lengths['video'][args.video_feats]
        args.audio_seq_len = max_seq_lengths['audio'][args.audio_feats]

        args.text_feat_dim = feat_dims['text'][args.text_backbone]
        args.video_feat_dim = feat_dims['video'][args.video_feats]
        args.audio_feat_dim = feat_dims['audio'][args.audio_feats]

        self.label_list = bm["intent_labels"]"""

# if문을 추가하여 텍스트 전용 모델일 경우 비디오/오디오 설정을 건너뛰도록
# 수정된 __init__ 함수 정의를 만듭니다.
replacement_init_def = """    def __init__(self, args):

        self.logger = logging.getLogger(args.logger_name)

        bm = benchmarks[args.dataset]
        max_seq_lengths = bm['max_seq_lengths']
        feat_dims = bm['feat_dims']

        args.text_seq_len = max_seq_lengths['text']
        args.text_feat_dim = feat_dims['text'][args.text_backbone]

        # 만약 실행하는 모델이 텍스트 전용 모델이 아니라면, 비디오/오디오 설정을 합니다.
        if args.method not in ['text', 'text_ood']:
            args.video_seq_len = max_seq_lengths['video'][args.video_feats]
            args.audio_seq_len = max_seq_lengths['audio'][args.audio_feats]
            args.video_feat_dim = feat_dims['video'][args.video_feats]
            args.audio_feat_dim = feat_dims['audio'][args.audio_feats]

        self.label_list = bm["intent_labels"]"""

# 파일 내용에서 원본 코드를 수정된 코드로 교체합니다.
# 이전 수정으로 인해 original_code가 약간 다를 수 있으므로, original_init_def를 찾아서 교체
if original_init_def in original_code:
    modified_code = original_code.replace(original_init_def, replacement_init_def)
else:
    # 만약 이전 수정으로 인해 정확히 일치하는 블록이 없다면, 각 라인을 개별적으로 처리
    print("블록 단위 수정 실패. 라인 단위 수정을 시도합니다...")
    lines = original_code.split('\\n')
    new_lines = []
    in_datamanager_init = False
    for line in lines:
        if "def __init__(self, args):" in line and "class DataManager" in "\n".join(new_lines[-2:]): # DataManager의 init인지 확인
            in_datamanager_init = True
            new_lines.append(line)
        elif "self.label_list = bm[\"intent_labels\"]" in line and in_datamanager_init:
            # 문제가 되는 라인들 대신 수정된 라인들을 추가
            new_lines.append("        # 만약 실행하는 모델이 텍스트 전용 모델이 아니라면, 비디오/오디오 설정을 합니다.")
            new_lines.append("        if args.method not in ['text', 'text_ood']:")
            new_lines.append("            args.video_seq_len = max_seq_lengths['video'][args.video_feats]")
            new_lines.append("            args.audio_seq_len = max_seq_lengths['audio'][args.audio_feats]")
            new_lines.append("            args.video_feat_dim = feat_dims['video'][args.video_feats]")
            new_lines.append("            args.audio_feat_dim = feat_dims['audio'][args.audio_feats]")
            new_lines.append(line)
            in_datamanager_init = False
        elif in_datamanager_init and ("video_seq_len" in line or "audio_seq_len" in line or "video_feat_dim" in line or "audio_feat_dim" in line):
            # 문제가 되는 라인들은 건너뜀
            continue
        else:
            new_lines.append(line)
    modified_code = "\\n".join(new_lines)


with open(target_file_data_base, 'w', encoding='utf-8') as f:
    f.write(modified_code)

print("\n패치가 완료되었습니다!")
print("이제 'run_text_ood_baselines.sh' 또는 관련 python 명령어를 다시 실행하시면 됩니다.")

# (선택적 검증) 수정이 잘 되었는지 확인
print("\n--- 수정된 DataManager 클래스 __init__ 부분 ---")
!grep -A 12 "class DataManager:" data/single_turn/base.py
print("---------------------------------------------")

KeyError 재발생 해결을 위해 'data/single_turn/base.py' 파일을 다시 패치합니다...
블록 단위 수정 실패. 라인 단위 수정을 시도합니다...

패치가 완료되었습니다!
이제 'run_text_ood_baselines.sh' 또는 관련 python 명령어를 다시 실행하시면 됩니다.

--- 수정된 DataManager 클래스 __init__ 부분 ---
class DataManager:
    
    def __init__(self, args):
        
        self.logger = logging.getLogger(args.logger_name)

        bm = benchmarks[args.dataset]
        max_seq_lengths = bm['max_seq_lengths']
        feat_dims = bm['feat_dims']

        args.text_seq_len = max_seq_lengths['text']
        args.video_seq_len = max_seq_lengths['video'][args.video_feats]
        args.audio_seq_len = max_seq_lengths['audio'][args.audio_feats]
---------------------------------------------


In [ ]:
!sh examples/single_turn/run_text_ood_baselines.sh

Streaming output truncated to the last 5000 lines.
Iteration:  19% 26/139 [00:00<00:02, 40.79it/s]
Iteration:  22% 31/139 [00:00<00:02, 42.61it/s]
Iteration:  26% 36/139 [00:01<00:02, 43.78it/s]
Iteration:  29% 41/139 [00:01<00:02, 44.66it/s]
Iteration:  33% 46/139 [00:01<00:02, 45.12it/s]
Iteration:  37% 51/139 [00:01<00:01, 45.35it/s]
Iteration:  40% 56/139 [00:01<00:01, 45.69it/s]
Iteration:  44% 61/139 [00:01<00:01, 45.91it/s]
Iteration:  47% 66/139 [00:01<00:01, 46.37it/s]
Iteration:  51% 71/139 [00:01<00:01, 46.93it/s]
Iteration:  55% 76/139 [00:01<00:01, 47.37it/s]
Iteration:  58% 81/139 [00:02<00:01, 47.45it/s]
Iteration:  62% 86/139 [00:02<00:01, 47.55it/s]
Iteration:  65% 91/139 [00:02<00:01, 47.79it/s]
Iteration:  69% 96/139 [00:02<00:00, 47.69it/s]
Iteration:  73% 101/139 [00:02<00:00, 47.88it/s]
Iteration:  76% 106/139 [00:02<00:00, 47.55it/s]
Iteration:  80% 111/139 [00:02<00:00, 47.27it/s]
Iteration:  83% 116/139 [00:02<00:00, 47.23it/s]
Iteration:  87% 121/139 [00:02<00

In [ ]:
!sh examples/single_turn/run_mult_ood_baselines.sh

Streaming output truncated to the last 5000 lines.
Iteration: 17it [00:14,  1.28it/s]
Iteration: 18it [00:14,  1.28it/s]
Iteration: 19it [00:15,  1.28it/s]
Iteration: 20it [00:16,  1.29it/s]
Iteration: 21it [00:17,  1.29it/s]
Iteration: 22it [00:17,  1.29it/s]
Iteration: 23it [00:18,  1.29it/s]
Iteration: 24it [00:19,  1.29it/s]
Iteration: 25it [00:20,  1.29it/s]
Iteration: 26it [00:20,  1.29it/s]
Iteration: 27it [00:21,  1.29it/s]
Iteration: 28it [00:22,  1.29it/s]
Iteration: 29it [00:23,  1.28it/s]
Iteration: 30it [00:24,  1.28it/s]
Iteration: 31it [00:24,  1.27it/s]
Iteration: 32it [00:25,  1.28it/s]
Iteration: 33it [00:26,  1.29it/s]
Iteration: 34it [00:27,  1.30it/s]
Iteration: 35it [00:27,  1.30it/s]
Iteration: 36it [00:28,  1.30it/s]
Iteration: 37it [00:29,  1.31it/s]
Iteration: 38it [00:30,  1.30it/s]
Iteration: 39it [00:30,  1.31it/s]
Iteration: 40it [00:31,  1.30it/s]
Iteration: 41it [00:32,  1.30it/s]
Iteration: 42it [00:33,  1.30it/s]
Iteration: 43it [00:34,  1.30it/s]
Iter

In [ ]:
!sh examples/single_turn/run_mag_bert_ood_baselines.sh

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Iteration: 100it [00:26,  3.88it/s]
Iteration: 101it [00:27,  3.87it/s]
Iteration: 102it [00:27,  3.85it/s]
Iteration: 103it [00:27,  3.85it/s]
Iteration: 104it [00:27,  3.86it/s]
Iteration: 105it [00:28,  3.88it/s]
Iteration: 106it [00:28,  3.88it/s]
Iteration: 107it [00:28,  3.88it/s]
Iteration: 108it [00:28,  3.88it/s]
Iteration: 109it [00:29,  3.88it/s]
Iteration: 110it [00:29,  3.88it/s]
Iteration: 111it [00:29,  3.88it/s]
Iteration: 112it [00:30,  3.89it/s]
Iteration: 113it [00:30,  3.89it/s]
Iteration: 114it [00:30,  3.89it/s]
Iteration: 115it [00:30,  3.89it/s]
Iteration: 116it [00:31,  3.88it/s]
Iteration: 117it [00:31,  3.88it/s]
Iteration: 118it [00:31,  3.88it/s]
Iteration: 119it [00:31,  3.88it/s]
Iteration: 120it [00:32,  3.88it/s]
Iteration: 121it [00:32,  3.87it/s]
Iteration: 122it [00:32,  3.87it/s]
Iteration: 123it [00:32,  3.87it/s]
Iteration: 124it [00:33,  3.89it/s]
Iteration: 125it [00:33,  3.89it/s]
Iteration: 126it [00:33,  3.

In [ ]:
!sh examples/single_turn/run_mag_bert_baselines.sh

Streaming output truncated to the last 5000 lines.
Iteration:  24% 92/386 [00:16<00:39,  7.40it/s]
Iteration:  24% 93/386 [00:16<00:39,  7.37it/s]
Iteration:  24% 94/386 [00:16<00:39,  7.33it/s]
Iteration:  25% 95/386 [00:16<00:39,  7.34it/s]
Iteration:  25% 96/386 [00:16<00:39,  7.36it/s]
Iteration:  25% 97/386 [00:16<00:39,  7.37it/s]
Iteration:  25% 98/386 [00:16<00:39,  7.38it/s]
Iteration:  26% 99/386 [00:17<00:38,  7.38it/s]
Iteration:  26% 100/386 [00:17<00:38,  7.38it/s]
Iteration:  26% 101/386 [00:17<00:38,  7.39it/s]
Iteration:  26% 102/386 [00:17<00:38,  7.39it/s]
Iteration:  27% 103/386 [00:17<00:38,  7.40it/s]
Iteration:  27% 104/386 [00:17<00:38,  7.39it/s]
Iteration:  27% 105/386 [00:17<00:38,  7.38it/s]
Iteration:  27% 106/386 [00:18<00:37,  7.37it/s]
Iteration:  28% 107/386 [00:18<00:38,  7.33it/s]
Iteration:  28% 108/386 [00:18<00:37,  7.34it/s]
Iteration:  28% 109/386 [00:18<00:37,  7.35it/s]
Iteration:  28% 110/386 [00:18<00:37,  7.36it/s]
Iteration:  29% 111/386 [0